In [86]:
from math import log, sqrt, exp
import datetime as dt                   # date objects
import numpy as np                      # array manipulation
import matplotlib.pyplot as plot        # plotting
import pandas as pd                     # data analysis
import pandas_datareader as pdr

from ib_insync import *
util.startLoop()

#### Trading Assignment 1

Go to https://finance.yahoo.com and download adjusted close daily prices (end of trading day prices adjusted for dividends and splits) for the period 03/15/2022 to 09/15/2022 for SPY ETF.

In [87]:
# pull data from 
source = 'yahoo'
indicators = ['SPY']
end_date = dt.date.today()
start_date = end_date - dt.timedelta(days=365)
raw_data = pdr.DataReader(indicators, source, start_date, end_date)
data = raw_data['Adj Close']

1. Calculate the annual volatility for the S&P500 index. Use 252 trading days.

In [88]:
std_SPY = data.tail(252).SPY.std()
print(f"Annual Volatility SPY: {std_SPY}")

Annual Volatility SPY: 25.696488913319122


2. Go to https://www.cboe.com/delayed_quotes/spy/quote_table and select five option contracts that are nearest to ATM with expiration as close as possible to 30 days and have open interest and volume of at least 1,000 contracts. Alternatively, connect to IB and download the option chain. (this part can be done with a bot)
Using the volatility, you calculated in (1) and the Black-Scholes model, compute the “theoretical” prices of these call options.

In [89]:
# initialize connection to IBKR
ib = IB()
# ib.connect('127.0.0.1', 7497, clientId=1)  # IB Trader Workstation
ib.connect('127.0.0.1', 4002, clientId=1)    # IB Gateway

<IB connected to 127.0.0.1:4002 clientId=1>

In [90]:
# create a contract for the underlying s&p500 index
spx = Index('SPX', 'CBOE')
ib.qualifyContracts(spx)
ib.reqMarketDataType(4)

# grab the ticker
[ticker] = ib.reqTickers(spx)
spx_price = ticker.marketPrice()

In [95]:
# generate the option chain
opt_chain = ib.reqSecDefOptParams(spx.symbol, '', spx.secType, spx.conId)
chain = next(c for c in opt_chain if c.tradingClass == 'SPX' and c.exchange == 'SMART')

# filter strikes for those close to in the money
strikes = [s for s in chain.strikes
           if spx_price - 20 < s < spx_price + 20]

expirations = sorted(dt.datetime.strptime(exp, "%Y%m%d") for exp in chain.expirations)
expiry = min(expirations, key=lambda x: abs(x - dt.timedelta(days=30)))

rights = ['C']

contracts = [Option('SPX', expiration, strike, right, 'SMART', tradingClass='SPX')
        for right in rights
        for expiration in expirations
        for strike in strikes]
contracts = ib.qualifyContracts(*contracts)

TypeError: bad operand type for abs(): 'datetime.datetime'

30

In [ ]:
# TODO now calculate the theoretical prices of these options using BS
# there might be a library out there that does this efficiently, but it's pretty easy to write

def d1(S,K,T,r,sigma):
    return (log(S/K) + (r+sigma**2/2.)*T) / (sigma*sqrt(T))

def d2(S,K,T,r,sigma):
    return d1(S,K,T,r,sigma) - sigma*sqrt(T)

def bs_call(S,K,T,r,sigma):
    return S*norm.cdf(d1(S,K,T,r,sigma)) - K*exp(-r*T)*norm.cdf(d2(S,K,T,r,sigma))
  
def bs_put(S,K,T,r,sigma):
    return K*exp(-r*T) - S*bs_call(S,K,T,r,sigma)



3. At the beginning of the first trading day, sell 10 contracts of the call option with the largest mispricing. What would the proceeds from your trade be if the market price equals the BS option value? What are the actual proceeds from your trade?

In [ ]:
# find the option with the largest mispricing compared to the strike

4. As soon as possible after executing the trade, you initiate a delta hedge for your options. The hedge will be rebalanced (adjusted) during the next four weeks, i.e., until the expiation date or until the hedged position is closed. \n At initiation of your hedge, what is the delta for the call option? How many shares do you have to trade? Considering the proceeds from the options sale, how much would you have to you have to borrow (in real life) to finance a delta-neutral position. What is the interest cost incurred at the end of the first trading day?

5. At least twice during each trading day for the next four weeks, rebalance your portfolio so that your risk exposure remains as close to delta neutral as possible. Remember transaction costs! Do not trade unless the stock has moved enough, i.e. more than a round-trip transaction costs!  (this part can be done with an algo)

6. In four weeks, the options are either exercised or expire with 0 value or the hedged position is closed. What is the value of your delta neutral position? Did you make or lost money? Explain why?